# Deep Dive: OpenAI API
Explore the key components of OpenAI API with a focus on Embeddings and Chat Completions.

## API Parts Overview
- **Embeddings:** Vector representations of text enabling semantic search, clustering, similarity.
- **Chat Completions:** Conversational AI responses using advanced LLMs like GPT-4o.
- **Text completions:** Traditional completion style prompts.
- Other features such as image generation, moderation are available but out of scope here.

# Embeddings
## What are Embeddings?
Embeddings are dense numerical vector representations of input text (or other data) capturing semantic relationships.
Texts with similar meanings produce vectors that are close in vector space.
- Used in similarity search, clustering, recommendations, semantic search, etc.

## OpenAI Embeddings API
**Input schema:**
- `model`: embedding model name (e.g., `text-embedding-3-small`)
- `input`: string or list of strings to embed

**Output schema:**
- `data`: list of objects containing `embedding` (vector of floats), `index`
- `usage`: tokens used information

Typical call: `client.embeddings.create(model, input)`

In [ ]:
# Embeddings creation example
import os
from openai import OpenAI

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
texts = [
    "Artificial intelligence is transforming the world.",
    "Machine learning models can classify images and text."
]
response = client.embeddings.create(
    model='text-embedding-3-small',
    input=texts
)

for i, data in enumerate(response.data):
    print(f"Text: {texts[i]}")
    print(f"Embedding vector (first 5 dimensions): {data.embedding[:5]}")

## Use Case: Text Summarization with Embeddings + Chat
We combine embeddings for document representations and chat completions for generating summaries.
1. Compute document embeddings
2. Use LLM chat to summarize or answer questions based on embeddings.

In [ ]:
# Sample documents
documents = [
    "OpenAI provides state-of-the-art language models.",
    "Embeddings allow semantic search and clustering.",
    "LangChain helps build chained LLM workflows."
]

# Get embeddings for documents
doc_embeddings_resp = client.embeddings.create(model='text-embedding-3-small', input=documents)
doc_embeddings = [d.embedding for d in doc_embeddings_resp.data]

# Combine embeddings with chat completions (simplified example)
query_text = "What does OpenAI provide?"

# Simple similarity check (cosine) - stub, actual code would implement fully
def cosine_sim(vec1, vec2):
    import numpy as np
    vec1, vec2 = np.array(vec1), np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

query_embedding_resp = client.embeddings.create(model='text-embedding-3-small', input=query_text)
query_embedding = query_embedding_resp.data[0].embedding

similarities = [cosine_sim(query_embedding, emb) for emb in doc_embeddings]
best_doc = documents[similarities.index(max(similarities))]

# Use chat completion to summarize the best matching document
chat_resp = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Summarize the following: {best_doc}"}
    ]
)

print("Summary:")
print(chat_resp.choices[0].message.content)

# Chat Completions
## What are Chat Completions?
Interactive, multi-turn conversations with language models optimized for chat.
Chat-based models understand roles like system, user, assistant.

## Schema Overview
**Input:**
- `model`: model name (e.g., 'gpt-4o')
- `messages`: list of dicts with `role` (system/user/assistant) and `content`
- Optional parameters: temperature, max tokens, stop sequences.

**Output:**
- `choices`: list with message outputs containing role and content
- `usage`: token usage counts

In [ ]:
# Basic Chat Completion example
chat_response = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": "Explain the concept of transfer learning in AI."}
    ],
    temperature=0.7,
    max_tokens=150
)

print(chat_response.choices[0].message.content)

## Use Case: Question Answering
Leverage chat completions to answer user questions interactively.
Built on top of document embeddings + semantic search, or external knowledge sources.

In [ ]:
# Simple Q&A example leveraging a document context
context = "OpenAI develops advanced AI models like GPT-4 and Codex."
question = "What advanced models has OpenAI developed?"

qa_chat = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {"role": "system", "content": "You are a knowledgeable assistant."},
        {"role": "user", "content": f"Answer based on this context: {context} \n Question: {question}"}
    ]
)

print("Answer:")
print(qa_chat.choices[0].message.content)